In [ ]:
import csv

In [ ]:
trainlog_lines = []

In [ ]:
#import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

images = []
measurements = []
correction = 0.1

def load_images(base_path):
    for line in trainlog_lines:
        file_pathname = line[0].split("/")[-1]
        image = plt.imread(base_path + file_pathname)
        images.append(image)
        steering_meas = float(line[3])
        measurements.append(steering_meas)

        l_file_pathname = line[1].split("/")[-1]
        image = plt.imread(base_path + l_file_pathname.lstrip())
        images.append(image)
        measurements.append(steering_meas + correction)
    
        r_file_pathname = line[2].split("/")[-1]
        image = plt.imread(base_path + r_file_pathname.lstrip())
        images.append(image)
        measurements.append(steering_meas - correction)

In [ ]:
# Load dataset 1
with open("../../data2/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    iter = 0
    for line in reader:
        trainlog_lines.append(line)
        iter += 1
    print (iter)
print (trainlog_lines[1])    
load_images("../../data2/IMG/")

In [ ]:
# Load dataset 2
trainlog_lines = []
with open("../../data/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    iter = 0
    for line in reader:
        trainlog_lines.append(line)
        iter += 1
    print (iter)
print (trainlog_lines[1])    
load_images("../../data/IMG/")

In [ ]:
# Load dataset 3
trainlog_lines = []
with open("../../data3/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    iter = 0
    for line in reader:
        trainlog_lines.append(line)
        iter += 1
    print (iter)
print (trainlog_lines[1])    
load_images("../../data3/IMG/")

In [ ]:
# Load dataset 4
trainlog_lines = []
with open("../../data4/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    iter = 0
    for line in reader:
        trainlog_lines.append(line)
        iter += 1
    print (iter)
print (trainlog_lines[1])    
load_images("../../data4/IMG/")

In [ ]:
# Load dataset 5
trainlog_lines = []
with open("../../data5/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    iter = 0
    for line in reader:
        trainlog_lines.append(line)
        iter += 1
    print (iter)
print (trainlog_lines[1])    
load_images("../../data5/IMG/")

In [ ]:
# Load dataset 6
trainlog_lines = []
with open("../../data6/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    iter = 0
    for line in reader:
        trainlog_lines.append(line)
        iter += 1
    print (iter)
print (trainlog_lines[1])    
load_images("../../data6/IMG/")

In [ ]:
aug_images, aug_meas = [], []
iter = 0
for image, meas in zip(images, measurements):
    aug_images.append(image)
    aug_meas.append(meas)
    aug_images.append(np.fliplr(image))
    aug_meas.append(-1.0*meas)
    iter += 1
    if (iter == 200):
        plt.imshow(image)
        plt.show()
        plt.imshow(np.fliplr(image))
        print (meas, -1.0*meas)

In [ ]:
X_train = np.array(aug_images)
y_train = np.array(aug_meas)

In [ ]:
print (X_train.shape)
print (y_train.shape)

In [ ]:
def show_loss_stats(history_object):
    ### print the keys contained in the history object
    print(history_object.history.keys())

    ### plot the training and validation loss for each epoch
    plt.plot(history_object.history['loss'])
    plt.plot(history_object.history['val_loss'])
    plt.title('model mean squared error loss')
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
    plt.legend(['training set', 'validation set'], loc='upper right')
    plt.show()

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Lambda, Conv2D, MaxPooling2D, Cropping2D, Dropout
from keras import optimizers
print (keras.__version__)

def lenet_model():
    print ('\nTraining using lenet_model()...\n')
    model = Sequential()
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D( cropping=((50,20), (0,0))) )
    model.add(Conv2D(6, (5,5) ,activation="relu"))
    model.add(MaxPooling2D())
    model.add(Conv2D(6, (5,5) ,activation="relu"))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(120))
    model.add(Dense(84))
    model.add(Dense(1))

    model.compile(loss="mse", optimizer="adam")
    history_object = model.fit(X_train,y_train,validation_split=0.2, shuffle=True, epochs=1, batch_size=512, verbose=2)

    model.save('model.h5')
    show_loss_stats(history_object)

    
def nvidia_model():
    print('\nTraining using Nvidia model...\n')
    model = Sequential()
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D( cropping=((50,20), (0,0))) )
    model.add(Conv2D(24, (5,5), strides=(2,2), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv2D(36, (5,5), strides=(2,2), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv2D(48, (5,5), strides=(2,2), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    
    model.compile(loss="mse", optimizer="adam")
#    opt = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
#    opt = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
#    model.compile(loss="mse", optimizer=opt)
    history_object = model.fit(X_train,y_train,validation_split=0.3, shuffle=True, epochs=8, batch_size=512, verbose=2)

    model.save('model.h5')
    show_loss_stats(history_object)
    
    
#lenet_model()    
nvidia_model()

